In [ ]:
from awe.data import swde, qa_dataset

In [ ]:
sds = swde.Dataset(suffix='-exact')
websites = sds.verticals[0].websites

In [ ]:
page = websites[0].pages[0]
page.file_path

In [ ]:
import selectolax

In [ ]:
tree = selectolax.parser.HTMLParser(page.html)

In [ ]:
tree.strip_tags([
    'script',
    'style',
    'head',
    '[document]',
    'noscript',
    'iframe'
])

In [ ]:
nodes_text = qa_dataset.collapse_whitespace(' '.join(qa_dataset.collapse_whitespace(n.text(deep=False)) for n in tree.body.traverse()))
body_text = qa_dataset.collapse_whitespace(tree.body.text(separator=' '))
len(nodes_text), len(body_text)

In [ ]:
import importlib
import pandas as pd
from awe import selectolax_utils, qa_model
_ = importlib.reload(selectolax_utils)

In [ ]:
pipeline = qa_model.QaPipeline()

In [ ]:
tokenizer = pipeline.tokenizer

In [ ]:
df = pd.DataFrame([
    {
        'xpath': selectolax_utils.get_xpath(node),
        'text': text,
        'tokens': len(tokenizer(text, add_special_tokens=False, return_attention_mask=False)['input_ids'])
    }
    for node in tree.body.traverse(include_text=True)
    if selectolax_utils.is_text(node) and not (text := qa_dataset.collapse_whitespace(node.text(deep=False))).isspace() and text != ''
])
df

In [ ]:
df['tokens'].sum()

In [ ]:
separate_tokens = [t for ts in tokenizer(list(df['text']), add_special_tokens=False, return_attention_mask=False)['input_ids'] for t in ts]
len(separate_tokens)

In [ ]:
split_words_tokens = tokenizer(list(df['text']), add_special_tokens=False, is_split_into_words=True)['input_ids']
len(split_words_tokens)

In [ ]:
separate_tokens == split_words_tokens

In [ ]:
len(tokenizer(df['text'].str.cat(sep=' '), add_special_tokens=False, return_attention_mask=False)['input_ids'])

In [ ]:
import awe.qa.collater
_ = importlib.reload(awe.qa.collater)

In [ ]:
collater = awe.qa.collater.QaCollater(tokenizer)

In [ ]:
collater([page])